<a href="https://colab.research.google.com/github/MarioROT/IHLT-MAI/blob/main/Session7_MarioRosas_AlamLopez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab session 7 (Word Sequences) - IHLT

**Students:**
- Mario Rosas
- Alam Lopez

**Lab Professor:** Salvador Medina Herrera

## Paraphrases Template

In [1]:
%%shell
git clone https://github.com/mariorot/IHLT-MAI.git
cd 'IHLT-MAI'
mv 'complementary_material' /content/
mv scripts /content/
pip install svgling
pip install python-crfsuite

Cloning into 'IHLT-MAI'...
remote: Enumerating objects: 387, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (144/144), done.
remote: Total 387 (delta 135), reused 89 (delta 53), pack-reused 190
Receiving objects: 100% (387/387), 324.75 KiB | 3.18 MiB/s, done.
Resolving deltas: 100% (225/225), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 993.5/993.5 kB 9.9 MB/s eta 0:00:00


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from scripts.compute_metrics import ComputeMetrics
from scripts.text_preprocessing import TextPreprocessing
from complementary_material.textserver import TextServer
from scripts.utils import pretty_table
from scipy.stats import pearsonr
import pandas as pd
pd.set_option("display.precision", 4)

[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet_ic.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet_ic to /root/nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


In [4]:
dt = pd.read_csv('complementary_material/test-gold/STS.input.SMTeuroparl.txt',sep='\t',header=None)
dt['gs'] = pd.read_csv('complementary_material/test-gold/STS.gs.SMTeuroparl.txt',sep='\t',header=None)

## TODO
1. Read all pairs of sentences of the SMTeuroparl files of test set within the evaluation framework of the project.

2. Compute their similarities by considering the following approach:

  - words plus NEs and Jaccard coefficient
  - ex: word_and_NEs=['John Smith', 'is', 'working']
  - Show the results.

Do you think it could be relevant to use NEs to compute the similarity between two sentences? Justify the answer.

In [ ]:
tp = TextPreprocessing()

# ----- Tokenization -----
# NLTK
dt[2] = tp.tokenize_data(list(dt[0]),'nltk')
dt[3] = tp.tokenize_data(list(dt[1]),'nltk')
# spaCy
dt[4] = tp.tokenize_data(list(dt[0]),'spacy')
dt[5] = tp.tokenize_data(list(dt[1]),'spacy')
# Data cleaning
dt[6]=tp.clean_data(list(dt[0]))
dt[7]=tp.clean_data(list(dt[1]))


# ----- Lemmatization -----
# -- With Tokens
# NLTK
dt[8]=tp.lemmatize_data(list(dt[0]),'nltk',False)
dt[9]=tp.lemmatize_data(list(dt[1]),'nltk',False)
# spaCy
dt[10]=tp.lemmatize_data(list(dt[0]),'spacy')
dt[11]=tp.lemmatize_data(list(dt[1]),'spacy')

# -- With Cleaned data
# NLTK
dt[12]=tp.lemmatize_data(list(dt[6]),'nltk')
dt[13]=tp.lemmatize_data(list(dt[7]),'nltk')
# spaCy
dt[14]=tp.lemmatize_data(list(dt[6]),'spacy')
dt[15]=tp.lemmatize_data(list(dt[7]),'spacy')


# ----- Word Desambiguation -----
# --- With Tokens
# NLTK
dt[16]= tp.wsd_lesk_data(list(dt[2]),'nltk', keep_failures=True, synset_word=True)
dt[17]= tp.wsd_lesk_data(list(dt[3]),'nltk', keep_failures=True, synset_word=True)
# Spacy
dt[18]= tp.wsd_lesk_data(list(dt[4]),'nltk', keep_failures=True, synset_word=True)
dt[19]= tp.wsd_lesk_data(list(dt[5]),'nltk', keep_failures=True, synset_word=True)
# Cleaned data
dt[20]= tp.wsd_lesk_data(list(dt[6]),'nltk', keep_failures=True, synset_word=True)
dt[21]= tp.wsd_lesk_data(list(dt[7]),'nltk', keep_failures=True, synset_word=True)

# --- With Lemmas
# NLTK Lemmas
dt[22]= tp.wsd_lesk_data(list(dt[8]),'nltk', keep_failures=True, synset_word=True)
dt[23]= tp.wsd_lesk_data(list(dt[9]),'nltk', keep_failures=True, synset_word=True)
# Spacy Lemmas
dt[24]= tp.wsd_lesk_data(list(dt[10]),'nltk', keep_failures=True, synset_word=True)
dt[25]= tp.wsd_lesk_data(list(dt[11]),'nltk', keep_failures=True, synset_word=True)
# Cleaned NLTK Lemmas
dt[26]= tp.wsd_lesk_data(list(dt[12]),'nltk', keep_failures=True, synset_word=True)
dt[27]= tp.wsd_lesk_data(list(dt[13]),'nltk', keep_failures=True, synset_word=True)
# Cleaned SpaCy Lemmas
dt[28]= tp.wsd_lesk_data(list(dt[14]),'nltk', keep_failures=True, synset_word=True)
dt[29]= tp.wsd_lesk_data(list(dt[15]),'nltk', keep_failures=True, synset_word=True)


# ----- Named Entities -----
# --- With Tokens
# NLTK
dt[30]= tp.named_entities_data(list(dt[0]), 'nltk', False)
dt[31]= tp.named_entities_data(list(dt[1]), 'nltk', False)
# SpaCy
dt[32]= tp.named_entities_data(list(dt[0]), 'spacy')
dt[33]= tp.named_entities_data(list(dt[1]), 'spacy')
# Cleaned data
# NLTK
dt[34]= tp.named_entities_data(list(dt[6]), 'nltk')
dt[35]= tp.named_entities_data(list(dt[7]), 'nltk')
# SpaCy
dt[36]= tp.named_entities_data(list(dt[6]), 'spacy')
dt[37]= tp.named_entities_data(list(dt[7]), 'spacy')

# --- With Lemmas
# NLTK
dt[38]= tp.named_entities_data(list(dt[8]), 'nltk')
dt[39]= tp.named_entities_data(list(dt[9]), 'nltk')

dt[40]= tp.named_entities_data(list(dt[8]), 'spacy')
dt[41]= tp.named_entities_data(list(dt[9]), 'spacy')
# SpaCy
dt[42]= tp.named_entities_data(list(dt[10]), 'nltk')
dt[43]= tp.named_entities_data(list(dt[11]), 'nltk')

dt[44]= tp.named_entities_data(list(dt[10]), 'spacy')
dt[45]= tp.named_entities_data(list(dt[11]), 'spacy')
# Cleaned data
# NLTK
dt[46]= tp.named_entities_data(list(dt[12]), 'nltk')
dt[47]= tp.named_entities_data(list(dt[13]), 'nltk')

dt[48]= tp.named_entities_data(list(dt[12]), 'spacy')
dt[49]= tp.named_entities_data(list(dt[13]), 'spacy')
# SpaCy
dt[50]= tp.named_entities_data(list(dt[14]), 'nltk')
dt[51]= tp.named_entities_data(list(dt[15]), 'nltk')

dt[52]= tp.named_entities_data(list(dt[14]), 'spacy')
dt[53]= tp.named_entities_data(list(dt[15]), 'spacy')

The code for the named entities extraction is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/text_preprocessing.py

The code for jaccard distance computation is at: https://github.com/MarioROT/IHLT-MAI/blob/main/scripts/compute_metrics.py

In [ ]:
# -- Jaccard computation
jaccard_pairs = {'jaccard_nltk_token':[2,3], 'jaccard_spacy_token':[4,5], 'jaccard_clean_token':[6,7], # Tokens
                 'jaccard_nltk_lemma':[8,9], 'jaccard_spacy_lemma':[10,11], 'jaccard_clean_nltk_lemma':[12,13], 'jaccard_clean_spacy_lemma':[14,15], # Lemmas
                 'jaccard_nltk_token_wsd':[16,17], 'jaccard_spacy_token_wsd':[18,19], 'jaccard_clean_token_wsd':[20,21], # WSD
                 'jaccard_nltk_lemma_wsd':[22,23], 'jaccard_spacy_lemma_wsd':[24,25], 'jaccard_clean_nltk_lemma_wsd':[26,27], 'jaccard_clean_spacy_lemma_wsd':[28,29], #WSD
                 'jaccard_nltk_ne':[30,31],  'jaccard_spacy_ne':[32,33],  'jaccard_clean_nltk_ne':[34,35],  'jaccard_clean_spacy_ne':[36,37], #Named Entities
                 'jaccard_nltk_lemmas_-_nltk_ne':[38,39],  'jaccard_nltk_lemmas_-_spacy_ne':[40,41],  'jaccard_spacy_lemmas_-_nltk_ne':[42,43],  'jaccard_spacy_lemmas_-_spacy_ne':[44,45], #Named Entities
                 'jaccard_clean_nltk_lemmas_-_nltk_ne':[46,47],  'jaccard_clean_nltk_lemmas_-_spacy_ne':[48,49],  'jaccard_clean_spacy_lemmas_-_nltk_ne':[50,51],  'jaccard_clean_spacy_lemmas_-_spacy_ne':[52,53] #Named Entities
                 }

for name, values in jaccard_pairs.items():
  dt[name] = ComputeMetrics(dt[values].to_numpy(), ['jaccard'], 1).do()['jaccard']

Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...
Computing jaccard...


In [ ]:
dt[['gs',*[col for col in dt.columns.to_list() if 'jaccard' in str(col)]]]

,gs,jaccard_nltk_token,jaccard_spacy_token,jaccard_clean_token,jaccard_nltk_lemma,jaccard_spacy_lemma,jaccard_clean_nltk_lemma,jaccard_clean_spacy_lemma,jaccard_nltk_token_wsd,jaccard_spacy_token_wsd,...,jaccard_clean_nltk_ne,jaccard_clean_spacy_ne,jaccard_nltk_lemmas_-_nltk_ne,jaccard_nltk_lemmas_-_spacy_ne,jaccard_spacy_lemmas_-_nltk_ne,jaccard_spacy_lemmas_-_spacy_ne,jaccard_clean_nltk_lemmas_-_nltk_ne,jaccard_clean_nltk_lemmas_-_spacy_ne,jaccard_clean_spacy_lemmas_-_nltk_ne,jaccard_clean_spacy_lemmas_-_spacy_ne
0,4.500,0.6538,0.6538,0.6923,0.6538,0.6000,0.6667,0.6667,0.7037,0.7037,...,0.6923,0.6923,0.6538,0.6538,0.6000,0.6000,0.6667,0.6667,0.6667,0.6667
1,5.000,0.2143,0.2143,0.2857,0.0769,0.0769,0.0000,0.0000,0.0769,0.0769,...,0.2857,0.2857,0.0769,0.0769,0.0769,0.0769,0.0000,0.0000,0.0000,0.0000
2,4.250,0.6087,0.6087,0.7273,0.6087,0.5455,0.7273,0.7273,0.6087,0.6087,...,0.7273,0.7273,0.6087,0.6087,0.5455,0.5455,0.7273,0.7273,0.7273,0.7273
3,4.500,0.4545,0.4000,0.2500,0.4545,0.4000,0.2500,0.2500,0.4545,0.4000,...,0.2500,0.2500,0.4545,0.4000,0.4000,0.3333,0.2500,0.2500,0.2500,0.2500
4,5.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,5.000,0.4500,0.4500,0.6364,0.4500,0.4500,0.6364,0.6364,0.5909,0.5909,...,0.6364,0.6364,0.4500,0.4500,0.4500,0.4500,0.6364,0.6364,0.6364,0.6364
455,4.750,0.6429,0.7143,0.2500,0.6429,0.6154,0.2500,0.2500,0.5385,0.6154,...,0.2500,0.2500,0.6429,0.7143,0.6154,0.5833,0.2500,0.2500,0.2500,0.2500
456,5.000,0.6000,0.6000,0.6667,0.6000,0.5263,0.6667,0.6667,0.6000,0.6000,...,0.6667,0.6667,0.6000,0.6316,0.5263,0.5556,0.6667,0.6667,0.6667,0.6667
457,4.000,0.6818,0.6818,0.8000,0.6818,0.6190,0.8000,0.8000,0.6818,0.6818,...,0.8000,0.8000,0.6818,0.6818,0.6190,0.6190,0.8000,0.8000,0.8000,0.8000


In [ ]:
pearson_group = lambda group: [round(pearsonr(dt['gs'], 1-dt[column])[0], 3) for column in dt.columns if group == str(column).split('_')[-1]]
names_group = lambda group: [' '.join(column.split('_')[1:-1]).upper() for column in dt.columns if group == str(column).split('_')[-1]]

results = {'Tokenization': {'Category':names_group('token'),'Values':pearson_group('token')},
           'Lemmatization': {'Category':names_group('lemma'), 'Values':pearson_group('lemma')},
           'Word Sense Disambiguation': {'Category':names_group('wsd'), 'Values':pearson_group('wsd')},
           'Named Entities': {'Category':names_group('ne'), 'Values':pearson_group('ne')}}

for name, dic in results.items():
  pretty_table(dic, name)


--- Results for Tokenization
+----------+--------+
| Category | Values |
+----------+--------+
|   NLTK   |  0.45  |
|  SPACY   | 0.461  |
|  CLEAN   | 0.468  |
+----------+--------+

--- Results for Lemmatization
+-------------+--------+
|   Category  | Values |
+-------------+--------+
|     NLTK    | 0.449  |
|    SPACY    | 0.477  |
|  CLEAN NLTK | 0.481  |
| CLEAN SPACY | 0.491  |
+-------------+--------+

--- Results for Word Sense Disambiguation
+-------------------+--------+
|      Category     | Values |
+-------------------+--------+
|     NLTK TOKEN    |  0.42  |
|    SPACY TOKEN    | 0.429  |
|    CLEAN TOKEN    |  0.48  |
|     NLTK LEMMA    | 0.417  |
|    SPACY LEMMA    | 0.422  |
|  CLEAN NLTK LEMMA | 0.482  |
| CLEAN SPACY LEMMA | 0.493  |
+-------------------+--------+

--- Results for Named Entities
+----------------------------+--------+
|          Category          | Values |
+----------------------------+--------+
|            NLTK            |  0.44  |
|        

# Conclusion

Until now we have tried to compute similarity of the STS task applying different levels of text preprocessing chaining different process as the preprocessing becomes more complex:

1.   Tokenization\
    a. NLTK tokenizer\
    b. spaCy tokenizer\
    c. Manual cleaning process
2.  Lemmatization\
    a. NLTK tokenizer + NLTK lemmatizer\
    b. spaCy tokenizer + spaCy lemmatizer\
    c. Manual cleaning + NLTK lemmatizer\
    d. Manual cleaning + spaCy lemmatizer
3. Word Sense Disambiguation (WSD)\
    a. NLTK tokenizer + WSD NLTK\
    b. spaCy tokenizer + WSD NLTK\
    c. Manual Cleaning + WSD NLTK\
    d. NLTK tokenizer + NLTK lemmatizer + WSD NLTK\
    e.spaCy tokenizer + spaCy lemmatizer + WSD NLTK\
    f. Manual cleaning + NLTK lemmatizer + WSD NLTK\
    g. Manual cleaning + spaCy lemmatizer + WSD NLTK
4. Word Sequences - Named Entities (NE) < ----------------------------------- THIS LAB SESION\
    a. NLTK tokenizer + NE NLTK\
    b. spaCy tokenizer + NE spaCy\
    c. Manual cleaning  + NE NLTK\
    d. Manual cleaning + Ne spaCy\
    e. NLTK tokenizer + NLTK lemmatizer + NE NLTK\
    f. NLTK tokenizer + NLTK lemmatizer + NE spaCy\
    g. spaCy tokenizer + spaCy lemmatizer + NE NLTK\
    h. spaCy tokenizer + spaCy lemmatizer + NE spaCy\
    i. Manual cleaning + NLTK lemmatizer + NE NLTK\
    j. Manual cleaning + NLTK lemmatizer + NE spaCy\
    k. Manual cleaning + spaCy lemmatizer + NE NLTK\
    l. Manual cleaning + spaCy lemmatizer + NE spaCy\



In this last excercise we extracted the named entities (with NLTK and spaCy implementations) and joined them as a single entity within the sentences. We applied these to the resulting sentences of previous processing techniques.

Furthermore, we enhanced our Word Sense Disambiguation (WSD) implementation from the preceding lab. We introduced the capability to decide whether to retain words for which the WSD technique is inapplicable. Additionally, we incorporated the option to select between directly comparing the synset objects or solely the word substrings within the synset. (Thus, better results were obtained in the similarity measures when applying this technique.)


It appears that using the "CLEAN" approach, which includes lowercasing, removing stopwords, punctuation, and short words, led to the highest STS values in general. This suggests that effective text cleaning and preprocessing can improve semantic similarity measurements.


Generally, the use od lemmatization improved results compared to tokenization alone. Throughout most of the sessions, the approach using SpaCy's lemmatization achieved the highest similarity values, which suggests that SpaCy's lemmatization method may be more effective for the data .


The results of applying WSD, after the last modification, seem to be less consistent in terms of improvement, with some cases performing worse. "CLEAN SPACY LEMMA" with manual cleaning, SpaCy's lemmatization and word sense disambiguation showed the highest values, indicating that combining cleaning, lemmatization with WSD can be effective for improving STS.


In this lab session applying the Word Sequences technique, the results  show that its impact can be mixed, with some combinations performing better than others. Again, using manual cleaning + spaCy lemmas produced the highest STS values in the named entities category. However, in all cases, the NLTK's named entity extraction and merging process obtained higher values.
It's worth noting that in some cases, the word sequences technique, may enhance similarity measurements, while in others, it might not provide significant benefits.

According to the increase in the value of the Pearson correlation as the techniques are chained we can verify that effective text preprocessing is crucial for improving semantic textual similarity measurements. The overall results continue to support the application of manual cleaning + spaCy lemmas for more complex techniques. While the best NE result did not outperform the best WSD result (with the latest update), bot areh effective for improving our similarity measurement process. However, the choice of the best preprocessing pipeline may depend on the specific goal.It is concluded then, that the unification of named entities can be valuable, but its impact may vary depending on the preprocessing pipeline chosen.



